# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sys
import re
import pickle
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_fscore_support,accuracy_score,label_ranking_average_precision_score
from sklearn.model_selection  import GridSearchCV
from sklearn.externals import joblib

nltk.download(['punkt','stopwords','wordnet'])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
[nltk_data] <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED]
[nltk_data]     certificate verify failed (_ssl.c:833)>


False

In [2]:
# load data from database

def load_data(db_path='data/DisasterResponse.db',tablename='disastertab'):
    """
    Function: load data from database and return X and y.
    Args:
      db_path(str): database file name included path
      tablename:(str): table name in the database file.
    Return:
      X(pd.DataFrame): messages for X
      y(pd.DataFrame): labels part in messages for y
    """
    # load data from database
    engine = create_engine('sqlite:///'+db_path)
    df=pd.read_sql_table(tablename, engine)
    X = df['message']
    # result is multiple classifty.
    y = df.iloc[:,4:]
    return X, y

In [3]:
X,y=load_data()

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):

    text = re.sub(r'[^a-zA-Z0-9]',' ',text.lower())
    

    words = word_tokenize(text)
    tokens = [w for w in words if w not in stopwords.words("english")]
    

    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect',TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=200)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=42)
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                 

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
def display_results(Y_test, y_pred):
    
    result=precision_recall_fscore_support(Y_test, y_pred)
    for i, col in enumerate(Y_test.columns.values):
        accu=accuracy_score(Y_test.loc[:,col],y_pred[:,i])
        score = ('{}\n Accuracy:  {:.4f}   % Precision: {:.4f}   % Recall {:.4f} '.format(
                  col,accu,result[0][i],result[1][i]))
        print(score)
    avg_precision = label_ranking_average_precision_score(Y_test, y_pred)
    avg_score= ('label ranking average precision: {}'.format(avg_precision))
    print(avg_score)

In [8]:
y_pred=pipeline.predict(X_test)

In [9]:
display_results(y_test, y_pred)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


related
 Accuracy:  0.9936   % Precision: 0.9949   % Recall 0.9968 
request
 Accuracy:  0.9945   % Precision: 0.9965   % Recall 0.9708 
offer
 Accuracy:  0.9998   % Precision: 1.0000   % Recall 0.9589 
aid_related
 Accuracy:  0.9905   % Precision: 0.9916   % Recall 0.9853 
medical_help
 Accuracy:  0.9971   % Precision: 0.9983   % Recall 0.9651 
medical_products
 Accuracy:  0.9967   % Precision: 0.9933   % Recall 0.9402 
search_and_rescue
 Accuracy:  0.9989   % Precision: 0.9953   % Recall 0.9658 
security
 Accuracy:  0.9990   % Precision: 0.9730   % Recall 0.9730 
military
 Accuracy:  0.9985   % Precision: 0.9980   % Recall 0.9579 
child_alone
 Accuracy:  1.0000   % Precision: 0.0000   % Recall 0.0000 
water
 Accuracy:  0.9986   % Precision: 0.9990   % Recall 0.9789 
food
 Accuracy:  0.9984   % Precision: 0.9949   % Recall 0.9909 
shelter
 Accuracy:  0.9983   % Precision: 0.9956   % Recall 0.9846 
clothing
 Accuracy:  0.9992   % Precision: 0.9875   % Recall 0.9634 
money
 Accuracy:  0.

### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
parameters = {
    'clf__estimator__min_samples_leaf': [1,10],
    'clf__estimator__max_features': ['auto','log2'],
    'vect__smooth_idf':[True]
}

cv = GridSearchCV(pipeline, param_grid=parameters,n_jobs=-1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [21]:
display_results(y_test, y_pred)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


related
 Accuracy:  0.9936   % Precision: 0.9949   % Recall 0.9968 
request
 Accuracy:  0.9945   % Precision: 0.9965   % Recall 0.9708 
offer
 Accuracy:  0.9998   % Precision: 1.0000   % Recall 0.9589 
aid_related
 Accuracy:  0.9905   % Precision: 0.9916   % Recall 0.9853 
medical_help
 Accuracy:  0.9971   % Precision: 0.9983   % Recall 0.9651 
medical_products
 Accuracy:  0.9967   % Precision: 0.9933   % Recall 0.9402 
search_and_rescue
 Accuracy:  0.9989   % Precision: 0.9953   % Recall 0.9658 
security
 Accuracy:  0.9990   % Precision: 0.9730   % Recall 0.9730 
military
 Accuracy:  0.9985   % Precision: 0.9980   % Recall 0.9579 
child_alone
 Accuracy:  1.0000   % Precision: 0.0000   % Recall 0.0000 
water
 Accuracy:  0.9986   % Precision: 0.9990   % Recall 0.9789 
food
 Accuracy:  0.9984   % Precision: 0.9949   % Recall 0.9909 
shelter
 Accuracy:  0.9983   % Precision: 0.9956   % Recall 0.9846 
clothing
 Accuracy:  0.9992   % Precision: 0.9875   % Recall 0.9634 
money
 Accuracy:  0.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [22]:
pipeline = Pipeline([
    ('vect',TfidfVectorizer(tokenizer=tokenize)),
    ('clf',  MultiOutputClassifier(AdaBoostClassifier()))
])
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.float64'>, encoding='utf-8',
                   input='content', lowercase=True, max_df=1.0, max_features=None,
                   min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                   smooth_idf=True, stop_words=None, strip_accents=None,
                   sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x142ff1598>, use_idf=True,
                   vocabulary=None)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                                      base_estimator=None,
                                                      learning_rate=1.0,
                                                      n_estimators=50,
                                                      random_state=None),
                  

In [23]:
parameters = {
    'vect__smooth_idf': [True,False],
}
# create grid search object
cv = GridSearchCV(pipeline, param_grid=parameters,n_jobs=-1)
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)   
display_results(y_test, y_pred)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


related
 Accuracy:  0.7880   % Precision: 0.7916   % Recall 0.9808 
request
 Accuracy:  0.8940   % Precision: 0.7792   % Recall 0.5118 
offer
 Accuracy:  0.9951   % Precision: 0.3333   % Recall 0.0548 
aid_related
 Accuracy:  0.7617   % Precision: 0.7640   % Recall 0.6067 
medical_help
 Accuracy:  0.9313   % Precision: 0.6532   % Recall 0.2628 
medical_products
 Accuracy:  0.9602   % Precision: 0.7105   % Recall 0.3435 
search_and_rescue
 Accuracy:  0.9748   % Precision: 0.6693   % Recall 0.1936 
security
 Accuracy:  0.9800   % Precision: 0.3333   % Recall 0.0608 
military
 Accuracy:  0.9737   % Precision: 0.6847   % Recall 0.3870 
child_alone
 Accuracy:  1.0000   % Precision: 0.0000   % Recall 0.0000 
water
 Accuracy:  0.9662   % Precision: 0.7634   % Recall 0.6751 
food
 Accuracy:  0.9490   % Precision: 0.7944   % Recall 0.7348 
shelter
 Accuracy:  0.9486   % Precision: 0.7968   % Recall 0.5462 
clothing
 Accuracy:  0.9883   % Precision: 0.7214   % Recall 0.4106 
money
 Accuracy:  0.

### 9. Export your model as a pickle file

In [24]:
with open('classifer.pkl', 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.